In [2]:
import requests
import os,sys,glob,shutil
import zipfile
import earthaccess

#### Step 1: Define a data directory where all the auxillary data is saved
* It is convinient to save all data at a one location, in different folders
* By default, the notebook will try to assign that to a data folder in the Home directory
* If you have a preference, please provide that below

In [3]:
#prefered_data_dir = '/nobackupp16/sbhusha1/data/ #uncomment this and provide directory of your choice
prefered_data_dir = None # Comment this out if uncommenting the above
if prefered_data_dir == None:
    data_directory = os.path.expanduser("~/data/")
else:
    data_directory = prefered_data_dir

print(f"Data will be saved in {data_directory}")

# Create the directory
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
# save this to bash export
! export DATADIR=$data_directory

Data will be saved in /home/jovyan/data/


#### Step 2: Download RGI Shapefiles
* We will fetch RGI 15 shapefiles for our study area


In [3]:
rgi_zip_url =  "http://www.glims.org/RGI/rgi60_files/15_rgi60_SouthAsiaEast.zip"
rgi_shp_fn = os.path.join(data_directory,"15_rgi60_SouthAsiaEast.zip")
if os.path.exists(rgi_shp_fn):
    print(f"file {rgi_shp_fn} already downloaded")
else:
    response = requests.get(rgi_zip_url)
    #Check for 200
    if response.ok:
        print ('OK!')
    else:
        print ('Query failed')
        sys.exit()
    #Write to disk
    open(rgi_shp_fn, 'wb').write(response.content)
    print(f"file {rgi_shp_fn} saved")

file /home/jovyan/data/15_rgi60_SouthAsiaEast.zip already downloaded


In [4]:
rgi_directory = os.path.join(data_directory,'rgi15/')
with zipfile.ZipFile(rgi_shp_fn, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(rgi_directory)

#### Step 3: Download ice thickness files from Farinotti et al. 2019
* We will again download files for the region 15 for concensus estimates
* We will also download files for individual models for uncertainty estimates 


In [5]:
ice_thickness_farinotti_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/composite_thickness_RGI60-15.zip?sequence=17&isAllowed=y'
ice_thickness_farinotti_zip = os.path.join(data_directory,'composite_thickness_RGI60-15.zip')

 
if os.path.exists(ice_thickness_farinotti_zip):
    print(f"file {ice_thickness_farinotti_zip} already downloaded")
else:
    response = requests.get(ice_thickness_farinotti_url)
    #Check for 200
    if response.ok:
        print ('OK!')
    else:
        print ('Query failed')
        sys.exit()
    #Write to disk
    open(ice_thickness_farinotti_zip, 'wb').write(response.content)
    print(f"file {ice_thickness_farinotti_zip} saved")

file /home/jovyan/data/composite_thickness_RGI60-15.zip already downloaded


In [7]:
## The file above contains data of ice thickness for each glacier in the region as an individual tiff file, 
#which will take a long time to unzip everything.
# so will selectively unzip files for only the six glaciers in the study
rgi_id_list = ['RGI60-15.03473','RGI60-15.03733','RGI60-15.03734','RGI60-15.03743','RGI60-15.04121','RGI60-15.04045']
ice_thickness_farinotti_dir = os.path.join(data_directory,'icethickness_farinotti/')
with zipfile.ZipFile(ice_thickness_farinotti_zip, 'r') as zipObj:
    for rgi in rgi_id_list:
      thickness_fn = f'RGI60-15/{rgi}_thickness.tif'
      zipObj.extract(thickness_fn, 
                     path=ice_thickness_farinotti_dir)
      


####TODO: Individual model output download

#### Step 4: Download debris thickness files from Rounce et al. 2020 study
* We will download files from NSIDC website using the earthacess library
* This requires us to have a NASA EarthData login

In [8]:
auth = earthaccess.login()

In [9]:
bounding_box = [84.44955936858184,26.811496290752743,87.98311172730928,29.272110001382174] #of our study site
Query = earthaccess.granule_query().short_name('HMA_DTE').version("1").bounding_box(*bounding_box)
Query.hits()

916

In [10]:
debris_thickness_directory = os.path.join(data_directory,'rounce_debris_thickness/')
granules = Query.get_all()
files = earthaccess.download(granules, debris_thickness_directory)

 Getting 916 granules, approx download size: 0.02 GB


QUEUEING TASKS | :   0%|          | 0/3820 [00:00<?, ?it/s]

File HMA_DTE_15.09474_bins.csv already downloaded
File HMA_DTE_15.09474_hdopt_prms.csv already downloaded
File HMA_DTE_15.09474_hdts_m.tif already downloaded
File HMA_DTE_15.09474_manifest.txt already downloaded
File HMA_DTE_15.09474_meltfactor.tif already downloaded
File HMA_DTE_15.09354_bins.csv already downloaded
File HMA_DTE_15.09354_hdopt_prms.csv already downloaded
File HMA_DTE_15.09354_hdts_m.tif already downloaded
File HMA_DTE_15.09354_manifest.txt already downloaded
File HMA_DTE_15.09354_meltfactor.tif already downloaded
File HMA_DTE_15.09255_bins.csv already downloaded
File HMA_DTE_15.09255_hdopt_prms.csv already downloaded
File HMA_DTE_15.09361_bins.csv already downloaded
File HMA_DTE_15.09255_manifest.txt already downloaded
File HMA_DTE_15.09255_meltfactor.tif already downloaded
File HMA_DTE_15.09255_hdts_m.tif already downloaded
File HMA_DTE_15.09361_hdopt_prms.csv already downloaded
File HMA_DTE_15.09361_hdts_m.tif already downloaded
File HMA_DTE_15.09361_manifest.txt alr

PROCESSING TASKS | :   0%|          | 0/3820 [00:00<?, ?it/s]

File HMA_DTE_15.03559_manifest.txt already downloaded
File HMA_DTE_15.04276_hdopt_prms_extrap.csv already downloaded
File HMA_DTE_15.03538_hdts_m_extrap.tif already downloaded
File HMA_DTE_15.04276_hdts_m_extrap.tif already downloaded
File HMA_DTE_15.03538_meltfactor_extrap.tif already downloaded
File HMA_DTE_15.03538_hdopt_prms_extrap.csv already downloaded
File HMA_DTE_15.04276_manifest.txt already downloaded
File HMA_DTE_15.03505_manifest.txt already downloaded
File HMA_DTE_15.03505_hdts_m_extrap.tif already downloaded
File HMA_DTE_15.03559_meltfactor_extrap.tif already downloaded
File HMA_DTE_15.03538_manifest.txt already downloaded
File HMA_DTE_15.03505_meltfactor_extrap.tif already downloaded
File HMA_DTE_15.04198_hdopt_prms_extrap.csv already downloaded
File HMA_DTE_15.03505_hdopt_prms_extrap.csv already downloaded
File HMA_DTE_15.04198_manifest.txt already downloaded
File HMA_DTE_15.04198_meltfactor_extrap.tif already downloaded
File HMA_DTE_15.04198_hdts_m_extrap.tif already d

COLLECTING RESULTS | :   0%|          | 0/3820 [00:00<?, ?it/s]

#### Step 5: Download Millan ice thickness files
* We will download files from NSIDC website using the earthacess library
* This requires us to have a NASA EarthData login

In [12]:

## Create data link from here: https://www.sedoo.fr/theia-publication-products/?uuid=55acbdd5-3982-4eac-89b2-46703557938c 
ice_thickness_millan_url = 'https://services-theia.sedoo.fr/glaciers/data/ige/v1_0/download/35021cd3-2eba-418c-9ead-829bfa24bf72'
ice_thickness_millan_zip = os.path.join(data_directory,'RGI-13-15.zip')

 
if os.path.exists(ice_thickness_millan_zip):
    print(f"file {ice_thickness_millan_zip} already downloaded")
else:
    response = requests.get(ice_thickness_millan_url)
    #Check for 200
    if response.ok:
        print ('OK!')
    else:
        print ('Query failed')
        sys.exit()
    #Write to disk
    open(ice_thickness_millan_zip, 'wb').write(response.content)
    print(f"file {ice_thickness_millan_zip} saved")

OK!
file /home/jovyan/data/RGI-13-15.zip saved


In [14]:
millan_thickness_directory = os.path.join(data_directory,'millan_icethickness/')
with zipfile.ZipFile(ice_thickness_millan_zip, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(millan_thickness_directory)

OSError: [Errno 28] No space left on device

In [ ]:
surface_velocity_millan_url = 'https://services-theia.sedoo.fr/glaciers/data/ige/v1_0/download/4100ce38-31e2-4c11-9be4-f14827728faa'
surface_velocity_millan_zip = os.path.join(data_directory,'RGI-13-15_velocity.zip')

if os.path.exists(surface_velocity_millan_zip):
    print(f"file {surface_velocity_millan_zip} already downloaded")
else:
    response = requests.get(surface_velocity_millan_url)
    #Check for 200
    if response.ok:
        print ('OK!')
    else:
        print ('Query failed')
        sys.exit()
    #Write to disk
    open(surface_velocity_millan_zip, 'wb').write(response.content)
    print(f"file {surface_velocity_millan_zip} saved")

In [ ]:
millan_velocity_directory = os.path.join(data_directory,'millan_velocity/')
with zipfile.ZipFile(surface_velocity_millan_zip, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(millan_velocity_directory)